Plans:

- Use a matrix to represent a state
- Map a matrix to another matrix (Do we have to do this? Is center-radius sufficient?)

Speed:
- If the fastest moves 1 grid step per time step (Others moving at a slower speed), everything can be resolved. Faster speed is essentially equivalent to larger gaps.

Event-handler:
- which two objects collide first?
- Build a loop network


GANs available or not?
- GAN provides an initial image and a final image
- Descriminator tells whether it is true or not?? (By a threshold?)

In [ ]:
class Ball(object):

    def _set_speed(self, speed_x, speed_y = 0):
        self.speed_x = speed_x
        self.speed_y = speed_y

    def _set_position(self,x,y):
        self.x=x
        self.y=y
    
    def __init__(self, x, speed_x, r, color, name, verbose, y = 50, speed_y = 0):
        self.name = name
        self.x = x
        self.y = y
        self._set_speed(speed_x,speed_y)
        self.r = r
        self.color = color
        self.mass=r**3
        self.verbose = verbose

    def get_x(self):
        return self.x

    def get_y(self):
        return self.y

    def get_speed_x(self):
        return self.speed_x

    def get_speed_y(self):
        return self.speed_y

    def get_new_x(self,dt):
        return self.x+self.speed_x*dt

    def get_new_y(self,dt):
        return self.y+self.speed_y*dt

    def get_params(self):
        return [self.x,self.y,self.speed_x,self.speed_y]



    def get_energy(self):
        return 0.5*self.mass*(self.speed_x**2+self.speed_y**2)
     
    def get_ball_col_time(self, o):
        dist = np.abs(self.x - o.x)
        v1x = self.speed_x
        v2x = o.speed_x 

        t_cand = np.abs(dist-self.r-o.r)/np.abs(v1x-v2x)

        newx=self.get_new_x(t_cand)
        onewx=o.get_new_x(t_cand)

        if dist<np.abs(newx-onewx):
            return 1e9
        else:
            return t_cand

    def get_wall_col_time(self, box):
        vx = self.speed_x
        x = self.x

        tmx, tpx = [np.abs((box.x - x)/vx) -np.abs(self.r/vx), np.abs((box.x + box.width - x)/vx)-np.abs(self.r/vx)]

        if vx>0:
            return tpx, 1
        else:
            return tmx, 0

    def move(self,dt):
        self.x=self.get_new_x(dt)

    def move_ball(self, o, dt):
        self.x=self.get_new_x(dt)
        o.x=o.get_new_x(dt)
        mass=self.mass
        o_mass=o.mass
        m1 = mass
        m2 = o_mass
        if self.verbose:
            print('old speed',self.speed_x,self.speed_y)

        v1x = self.speed_x
        v2x = o.speed_x

        v1x, v2x = ((m1-m2)/(m1+m2)*v1x+(2*m2)/(m1+m2)*v2x,
            (2*m1)/(m1+m2)*v1x+(m2-m1)/(m1+m2)*v2x)

        # v1n, v2n = v2n, v1n
        self.speed_x = v1x

        o.speed_x = v2x    

    def move_wall(self, dt, wall_index):
        self.x=self.get_new_x(dt)
        self.speed_x = -self.speed_x